# 概要

风险建模是一个强大的工具，可用于识别和管理投资组合中的风险源。 在本文中，我们阐述了Quantopian开发的股权风险因子模型QuantPian Risk Model（QRM）的逻辑和实施，此模型分解和归类风险敞口，可用于任意股权投资策略。 通过定义风险源，可将剩余或剩余部分视为策略的“alpha”，或者投资策略的技术收益率。 结合其他一些工具，Quantopian使用QRM观察持有期间投资组合，来评估定量交易策略。

# 介绍

风险管理考虑的是我们为了获得未来收益愿意承担多少风险。 这个过程涉及：
+ 识别风险来源
+ 量化风险敞口
+ 确定这些风险敞口的影响
+ 制定缓解策略
+ 观察后续的表现并根据需要修改缓解策略

风险来自投资组合的未来损失和收益的不确定性，不同的个人和机构对风险的容忍度不同。 我们将投资组合在`T`时期的总风险量化为投资组合回报的标准差：$$\sigma = {\sqrt{{1 \over T} {\sum_{l=1}^r} (r_l - \bar{r})^2 }}$$ 此处：
+ $r_l$为时点$l$的收益率
+ $\bar{r}$为整个$T$周期内的平均收益率

这是风险的一个常见定义。它对称地处理收益和损失，并可用于评估每个级别的投资组合，从单个资产到投资组合本身。标准差也称为波动率，描述期望值相对于均值接近程度。一个常见的经验法则是，约68％的值位于平均值的一个标准偏差内，95％的值位于平均值的两个标准偏差内，99％的值位于平均值的三个标准偏差内。具有低标准差的观测值群体将包含大部分集中在总体平均值周围的个体观测值，而具有较高标准差的观测值群体可能包含更多的极值，无论是收益还是损失。这符合关于财务收益的常识。极端值与资产收益率剧烈波动伴生。因此，可能获取暴利，也可能一夜破产。

评估风险不仅仅是评估潜在损失。它使我们能够设置合理预期，对潜在投资作出明智的决策。量化与投资组合相关的风险来源可以揭示投资组合在多大程度上实际达到了规定的投资目标。例如，如果投资策略被描述为以市场和行业中等水平为目标，那么基础投资组合不应该通过长期持有某一个行业(如技术行业）而获得大部分回报。虽然这种策略可能会在特定时间段内盈利，但了解这些利润是基于单个行业的非预期下注而获得的，可能会导致投资者对是否需要分配资金以及分配多少资金做出不同的决定。对风险敞口进行量化可让投资者和管理人员制定风险管理策略并完善其投资组合。


开发风险模型可以明确区分**共同风险**和**特定风险**。共同风险在这里被定义为归因于共同因子的风险，这些共同因子推动股票市场收益。这些因子可以由构成市场的基础投资资产的基本信息或统计信息组成。基本因子通常是发行股票的公司报告的可观察的基本比率，例如账面价值与股价的比率或每股收益。这些因子通常来自金融和宏观经济数据。统计因子使用数学模型来解释资产收益率时间序列之间的相关性，而不考虑公司特定的基础数据（Axioma，Inc.，2011）。


一些常用风险因子是整体市场指数的影响，如资本资产定价模型（CAPM）（Sharpe 1964），可归因于投资于个别行业的风险，这些风险可以为公司提供区间参考，如BARRA风险模型（BARRA，Inc.，1998），或者像Fama-French三因子模型那样模仿投资风格的主题因子，如投资“小盘股”公司或“高增长”公司。

**特定风险**在此定义为风险模型中包含的**共同风险因子**无法解释的风险。通常情况下，这是在考虑共同风险后留下的剩余部分（Axioma，Inc. 2011）。当在定量交易的背景下考虑风险管理时，我们对风险的理解在很大程度上用于阐明我们对“alpha”的定义。考虑投资组合的共同因子风险之后的这个残差可以被认为是投资组合α的代理或估计。

# 因子模型

通常，首次介绍因子模型及共同因子风险概念都会提及资本资产定价模型（CAPM）。 在CAPM中，我们定义了收益率与共同因子风险之间的均衡关系，它仅使用一个共同的因子风险即市场本身的回报。 CAPM表示任何单个资产的回报如下：$$E[r_i] = r_f + \beta^M_i (r_M - r_f)$$此处：
+ $r_f$为无风险利率
+ $r_M$为市场收益率
+ $\beta^M_i = {COV[r_i,r_m] \over VAR[r_M]}$是市场对第$i$个资产超额收益率的影响

CAPM将单个资产的收益分解为由整个市场解释的收益和由无风险资产预期的收益二部分。仅为市场分析的简化，在实证中并不存在。 CAPM自成立以来已经取得许多改进，如Fama-French三因子模型（Fama and French 1993），该模型从单一风险源扩展到另外两个风险源，公司规模和公司价值。Fama-French三因子模型可以表示为：$$r_i = r_f + \beta^M_i(r_M - r_f) + \beta^{SMB}_ir_{SMB} + \beta^{HML}_ir_{HML} + \alpha_i$$此处：
+ $r_{SMB}$是小市值股票相对于大市值股票（SMB）的风险溢价
+ $r_{HML}$是较高账面市场比率(HML)的股票相对于较低HML的风险溢价
+ $\beta^{SMB}_i$是$r_i$相对于SMB的风险敞口
+ $\beta^{HML}_i$是$r_i$相对于HML的风险敞口
+ $\alpha_i$是资产$i$的无法解释的收益

作为最后一个例子，套利定价理论（APT）（Ross，1976）是CAPM和类似模型的推广，它考虑多个因子驱动收益的影响。 APT使用多元线性回归（线性因子模型）表示单个资产的回报，如下所示：$$r_i = \alpha_i + \beta_{i,0}F_0 + \beta_{i,1}F_1 + ... + \beta_{i,m}F_m+ \epsilon_i$$此处：
+ $F_j,j \text{ } \epsilon \text { } \{0,m\}$是模型中共同因子的回报流
+ $\beta_{i,j},j \text{ } \epsilon \text { } \{0,m\}$是$r_i$对每个共同因子风险的风险敞口集合
+ $\alpha_i$是资产$i$的无法解释的收益
+ $\epsilon_i$是资产$i$的特质冲击

APT中的因子是完全由单一特征主导的收益率流。 在Fama-French模型中，我们解释收益的方法仅限于market，SMB和HML，而在APT中，我们可以添加尽可能多的因子，以解释与我们相关的各种常见因子。 APT构成了Quantopian风险模型的基础。

# 完成细节

QRM是一个多因子风险模型，旨在通过一套16个基本因子分解每项资产的回报。 模型中的16个因子由11个部门因子和5个主题因子组成。 QRM并未单独建立market模型因子。被选为共同风险因子的因子，其彼此独立的程度高，同时试图解释尽可能多的市场资产的回报。

QRM设计定义为模拟历史和当前风险，而不是作为风险预测工具。

本节开始QRM的技术实施。

## 符号指南

基本符号约定摘要：
+ 小写字母表示矢量，例如$a$
+ 大写字母表示矩阵，例如$A$
+ 带下标$\text{:,}j$的大写字母，例如$A_{:,j}$，即代表矩阵$A$的第$j$列
+ 带下标$i\text{,:}$的大写字母，例如$A_{i,:}$，表示矩阵$A$的第$i$行

## 数学模型

在数学上，QRM具有以下形式，

$$r_{i,t} = \sum_{j=1}^n\beta_{i,j,t}^{sect}f_{j,t}^{sect} + \sum_{k=1}^m\beta_{i,j,t}^{style}f_{j,t}^{style} + \epsilon_{i,t}, \tag1$$此处：
+ $r_{i,t}$是资产$i$在$t$日的收益
+ $n$是部门因子的数量
+ $m$是主题因子的数量
+ $\beta_{i,j,t}^{sect}$是资产$i$在第$t$天的第$j^{th}$个部门因子敞口。 因子敞口也被称为因子加载。它衡量因变量和潜在因子之间的关系。对于资产$i$，如果不属于$j^{th}$部门，则$\beta_{i,j,t}^{sect}$为零。
+ $f_{j,t}^{sect}$是第$t$天$j^{th}$部门因子的回归
+ $f_{k,t}^{style}$是第$t$天资产$i$的$k^{th}$主题因子风险
+ $\epsilon_{i,t}$是模型(1)中$t$天资产$i$的剩余部分。

数学模型(1)派生于子模型(1a)$$r_{i,t} = \sum_{j=1}^n\beta_{i,j,t}^{sect}f_{j,t}^{sect} + \epsilon_{i,t}^{sect},\tag{1a}$$和子模型(1b)$$\epsilon_{i,t}^{sect} = \sum_{k=1}^m\beta_{i,k,t}^{style}f_{k,t}^{style} + \epsilon_{i,t},\tag{1b}$$此处：
+ $\epsilon_{i,t}^{sect}$是子模型（1a)中$t$日资产$i$的剩余部分

## 部门因子

部门因子被用来表示不同部门的影响。 QRM根据Morningstar（Morningstar，Inc. n.d.）定义的行业部门分类定义部门。 此外，QRM使用部门ETF回报来表示相应的部门因子回报。 下表分别将数学模型（1）中的每个区段映射到其指数，相应的ETF，Morningstar区段代码，Quantopian安全标识符（SID）以及Quantopian API中使用的变量名称。

![部门因子](Quantopian Risk Model.png)

每个部门因子的收益都是已知的，Quantopian数据库中的每项资产都至多映射到一个单一部门。 因此，只有部门因子风险需要估计。

## 主题因子

QRM包括5种主题因子：动量，规模，价值，短期逆转和波动性。 每个主题因子都旨在复制传统的投资策略。 下表将数学模型（1）中的每个样式因子映射到其索引，以及Quantopian API中的变量名称：

![部门映射表](Quantopian Risk Model-1.png)

### 主题因子定义

动量：动量因子反映了在过去的11个月内上涨股票（赢家股票）和下跌股票（输家股票）之间的回报差异。

规模：规模因子反映了大盘股和小盘股之间的回报差异。

价值：价值因子捕捉"便宜的"股票和"贵的"股票之间的回报差异。

波动性：波动性因子捕捉高波动性股票和低波动性股票之间的回报差异。

短期逆转：短期逆转因子反映了近期损失严重理论上要扭转（最近的输家股票）和近期涨幅强劲理论上要扭转（最近的赢家股票）二类股票之间的回报差异。

### 度量公式

样式因子度量标准用于描述样式因子。下面，我们为QRM中的每个样式因子度量提供数学定义。

#### 动量 

在第$t$天，资产$i$（MOMENTUM）的动量指标通过计算12个月前至1个月前的11个月累计回报来计算。为避免预见偏差，所有样式因子指标都滞后一天。 公式是：$$MOMENTUM=\prod_{i=-c*12+t-1}^{-c+t-1}(1 + r_{i,l}),$$此处：
+ $r_{i,l,}$是第$l$天资产$i$的回报
+ $c$是一个月内的交易天数，这里设定为21的常数(*A股全年244天，接近*)

#### 公司规模

资产$i$的第t天`Size`指标为其公司市值的对数来。公式是：$$SIZE = log(M_{i, t-1})$$此处：
+ $M_{i,t-1}$是$t-1$日资产$i$的市值。Quantopian公司使用的财务数据是通过Pipeline API访问的晨星基本数据。

#### 价值指标

通过计算公司股东权益与市值之比，计算$t$日资产$i$的价值指标`VALUE`。 公式是：$$VALUE = { S_{i,t-1} \over M_{i,t-1}}$$此处：
+ S_{i,t-1}是公司资产$i$的股东权益

#### 短期反转

通过计算负相对强弱指数（RSI）计算t日资产i的短期反转指标STR。 公式是：$$STR = -1 * RSI_{t-1}$$此处：
+ $RSI_{t-1}$是从$t-1$日到$t-15$的14天时间范围内的相对强弱指数

#### 波动性

资产$i$在第$t$日的波动率通过计算6个月的收益波动率来计算。 公式是：$$VOL = \sqrt{{1 \over 6c}\sum_{l=t-1}^{-6c -1 + t}(r_{i,l} - \bar{r}_i)}$$此处：
+ $c$是一个月内的交易日数，这里设定为21。
+ $\bar{r}_i$是该时期$(-6c - 1 + t, t - 1)$资产$i$的平均收益

# 方法

如前所述，QRM由两个子模型组成。 子模型（1a）使用线性回归来估计所有股票的部门因子风险，并将剩余收益$\epsilon_{i,t}^{sect}$传递给子模型（1b）。 然后，子模型（1b）使用$\epsilon_{i,t}^{sect}$作为输入来估计与主题因子风险相关的主题因子回报。

## 部门因子计算

QRM使用尾部两年窗口的股票收益及其各自部门因子收益率来估算每种资产的行业因子风险。程序如下：

对于每个在第$t$天股票$i$:

1. 查找股票$i$的晨星行业代码
2. 选择与股票$i$的晨星行业代码相匹配的行业ETF
3. 计算股票$i$在第t天的2年尾部窗口历史收益，并将其格式化为矢量列$r_i$
4. 计算第2步所选ETF第$t$天的2年尾部窗口历史回报，并将其格式化为矢量列$f$
5. 在矢量列$f$上回归向量列$r_i$
6. 获得回归系数β，并将其设置为相应的部门因子风险敞口
7. 将其他部门因子风险敞口设置为零
8. 计算2年尾部窗口历史部门剩余收益。$\epsilon_i^{sect}=r_i - \beta f$

例如，令$t$等于2013年1月2日，股票$i$为`AAPL`。然后:
+ 选定的ETF为`XLK`
+ 矢量栏$f$为`XLK`从2010年12月31日到2013年1月1日的每日回报
+ 矢量列$r_i$为2010年12月31日到2013年1月1日的AAPL每日回报
+ $\beta$为AAPL的技术部门风险因子敞口
+ 矢量列$\epsilon_i^{sect}$等于$r_i - \beta f$，为2010年12月31日到2013年1月1日的历史部门回报率

如果我们将这个例子加到子模型（1a）中，它可以写成$$r_{i,t} = \sum_{j=1}^n\beta_{i,j,t}^{sect}f_{j,t}^{sect} + \beta f_t + \epsilon_{i,t}^{sect}$$此处：
+ $f_t$是$f$的最后一项
+ $\epsilon_{i,t}^{sect}$是$\epsilon_i^{sect}$的最后一项
+ $\sum_{j=1}^n\beta_{i,j,t}^{sect}f_{j,t}^{sect}$等于0

## 主题因子计算

为了估计主题因子的回报，在市场中使用所有股票是不合适的。 我们需要界定一个能代表市场的总体，估计总体，同时排除“有问题”的资产，如房地产投资信托基金，美国存托凭证，非流动性股票等。在估值领域选择股票是主观的。 QRM的估算领域有大约2100种股票。 选择标准包括：
+ 作为普通股
+ 有足够的数据来计算主题因子指标
+ 在流动性最高的3000股票中

估计范围之外的股票被称为互补股票。 包括估计总体中的股票和互补股票的总体被称为覆盖总体。 我们将演示如何计算估计领域中股票的主题因子风险，如何估计主题因子收益，以及如何计算互补股票的主题因子风险。

### 估计总体股票的主题因子风险敞口

第t天估计总体中股票的主题因子敞口为第t天股票主题因子的`Z分数`。它们是相对于估计总体的标准化数值(Z分数）。

### 估计主题因子收益率

使用横截面回归两年内每天估计主题因子回报。

在$t$天的尾部两年：
1. 计算估计总体中股票的5种主题因子风险敞口，并将它们存储在矩阵$B$的列中
2. 收集$t$日估计总体股票的部门残差，并将它们形成列向量$\epsilon_t^{sect}$
3. 在矩阵$B$上回归向量$\epsilon_t^{sect}$
4. 使用回归系数$f_{1,t}^{style}, f_{2,t}^{style}, ..., f_{5,t}^{style}$中获得5个主题因子收益率
5. 生成5个矢量列，$f_k^{style} (k = 1, 2, ... 5)$，然后将$f_{k,t}^{style}$存储在$f_k^{style}$
6. 从$\epsilon_t^{sect}$中减去$\sum_{k=1}^5B_{:k} f_{k,t}^{style}$，收集估计总体股票在第t天的剩余收益

图1以矩阵形式显示了$\epsilon_t^{sect}$，$\epsilon_i^{sect}$和$\epsilon_{i,t}^{sect}$之间的关系。

![图1](figure_1.png)

图2显示了$f_k^{style}$和$f_{kt}^{style}$之间的关系。

![图2](figure_2.png)

## 互补股票主题因子风险

通过求解5个带有部门残差时间序列主题因子多元线性回归来计算互补股票因子风险敞口。在这里，我们使用了一个2年尾部窗口的主题因子收益和部门剩余收益系列。 程序如下。
在$t$日，对每个互补股票$i$：
1. 收集主题因子收益率$f_k^{style}, k=1,2,...,5$
2. 收集尾部窗口2年行业历史剩余收益率$\epsilon_i^{sect}$
3. 获得回归系数$\beta_{k,t}^{style}k=1,2,...,5$并将它们设置为相应的主题因子风险
4. $\epsilon_i^{sect}$减去$\sum_{k=1}^5 \beta_{i,k,t}^{style} f_{k}^{style}$，计算2年历史残留收益率$\epsilon_i$

## 风险计算

资产$i$在$T$时段的风险定义为：$$\sigma = {\sqrt{{1 \over T} {\sum_{l=1}^r} (r_l - \bar{r}_i)^2 }}\tag2$$此处：
+ $r_{i,l}$为资产$i$在时间$l$的收益率
+ $\bar{r}_i$是资产$i$在$T$时段的平均收益

每个因子收益率的风险可以直接由等式(2)来计算。例如，整个$T$时段的第$k^{th}$主题因子风险为：$${\sqrt{{1 \over T} {\sum_{l=1}^T} (f_{k,l}^{style} - \bar{f}_k^{style})^2 }}$$

同样，每个加权因子收益率风险也可有等式(2)计算。例如，在$T$时间段内第$k^{th}$个加权主题因子敞口的风险是：$${\sqrt{{1 \over T} {\sum_{l=1}^T} (\beta_{i,k,t}^{style} f_{k,l}^{style} - \overline{\beta_k^{style} f_k^{style}}    )^2 }}.$$

# 总结和结论

Quantopian风险模型是一个16因子风险模型，用于帮助我们的用户和投资于高质量交易算法的研究和评估。 我们使用经典的金融技术来计算美国股票的每个相关因素的风险敞口。 在Quantopian Research环境和Quantopian网站的后台测试中，免费提供风险模型因素载荷和因子回报。 进一步的研究将出现在国际市场上可能有用的常见因素。

# 参考

Axioma, Inc. 2011. Axioma Robust Risk Model Handbook. Axioma, Inc.

BARRA, Inc. 1998. United States Equity. BARRA, Inc.

Fama, Eugene F, and Kenneth R French. 1993. "Common risk factors in the returns on stocks and bonds." Journal of Financial Economics 3-56.

Morningstar, Inc. n.d. Morningstar® Data for Equities.

Ross, Stephen A. 1976. "The Arbitrage Theory of Capital Asset Pricing." Journal of Economic Theory 341-360.

Sharpe, William F. 1964. "Capital Asset Prices: A Theory of Market Equilibrium under Conditions of Risk." The Journal of Finance 425-442.

# 附录

投资组合周转假设

用于QRM的数据的周期性是每天的，这导致假设每个资产的最小持有期至少为1天。 具有重大盘中交易或非常高周转率的投资策略不适合用当前的QRM进行分析。

投资覆盖

QRM覆盖美国股票市场约4000股，但并不包含所有资产。 如果投资组合在覆盖范围之外的资产上投入了相当大的资金，那么用QRM进行分析是不合适的。

目前的QRM不包括ETF和部分预选ETF（用于测试）。

![表3](t3.png)

# 下载PDF

白皮书可[在此下载](https://media.quantopian.com/quantopian_risk_model_whitepaper.pdf)